# OpenSky Arrivals Helper
This notebook demonstrates a helper that pulls today's arrivals for Boston Logan International Airport (BOS/KBOS) via the OpenSky API.


In [1]:
from datetime import datetime, timedelta, timezone
from pathlib import Path
from typing import Any, List, Optional
import os
import requests

import pandas as pd
from dotenv import load_dotenv

# Locate and load the nearest .env file (walk upwards from the notebook directory).
def _load_env() -> None:
    search_root = Path.cwd().resolve()
    for candidate in [search_root] + list(search_root.parents):
        env_path = candidate / '.env'
        if env_path.exists():
            print(os.environ.get('OPENSKY_USERNAME'))
            load_dotenv(env_path, override=True)
            return
    raise FileNotFoundError('No .env file found in notebook directory hierarchy.')


_load_env()
username = os.environ.get('OPENSKY_USERNAME')
password = os.environ.get('OPENSKY_PASSWORD')


None


In [5]:
AUTH_URL = "https://auth.opensky-network.org/auth/realms/opensky-network/protocol/openid-connect/token"

def get_auth_token(
    *,
    client_id: Optional[str] = None,
    client_secret: Optional[str] = None,
    timeout: float = 30.0,
) -> str:
    """Exchange OpenSky credentials for an access token using the client-credentials flow."""
    resolved_client_id = (client_id or os.environ.get("OPENSKY_CLIENT_ID") or username or "").strip()
    resolved_client_secret = (client_secret or os.environ.get("OPENSKY_CLIENT_SECRET") or password or "").strip()
    if not resolved_client_id or not resolved_client_secret:
        raise ValueError("Both client_id and client_secret are required to fetch an auth token.")

    payload = {
        "grant_type": "client_credentials",
        "client_id": resolved_client_id,
        "client_secret": resolved_client_secret,
    }

    response = requests.post(
        AUTH_URL,
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data=payload,
        timeout=timeout,
    )
    response.raise_for_status()
    token_response = response.json()
    try:
        return token_response["access_token"]
    except KeyError as exc:
        raise ValueError("OpenSky auth response did not include an access_token.") from exc


In [6]:
token = get_auth_token()

In [ ]:
OPEN_SKY_BASE_URL = "https://opensky-network.org/api"

def _ensure_timestamp(value: any) -> int:
    """Convert assorted timestamp inputs into the Unix epoch seconds OpenSky expects."""
    if isinstance(value, (int, float)):
        return int(value)
    converter = globals().get("datetime_to_unix")
    if converter is not None:
        return converter(value)  # Defer to the shared helper when available.
    if isinstance(value, datetime):
        dt = value
    elif isinstance(value, str):
        stripped = value.strip()
        if not stripped:
            raise ValueError("Timestamp string is empty.")
        normalized = stripped[:-1] + "+00:00" if stripped.endswith("Z") else stripped
        try:
            dt = datetime.fromisoformat(normalized)
        except ValueError as exc:
            raise ValueError(f"Unsupported timestamp string: {value}") from exc
    else:
        raise TypeError(f"Unsupported timestamp type: {type(value)!r}")
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    else:
        dt = dt.astimezone(timezone.utc)
    return int(dt.timestamp())

def get_departures(
    airport: str,
    begin: any,
    end: any,
    *,
    token_override: Optional[str] = None,
    session: Optional[requests.Session] = None,
    timeout: float = 30.0,
) -> List[dict]:
    """Fetch departure flights for an airport via the OpenSky REST endpoint."""
    if not airport:
        raise ValueError("airport is required")
    begin_ts = _ensure_timestamp(begin)
    end_ts = _ensure_timestamp(end)
    if end_ts <= begin_ts:
        raise ValueError("end must be after begin")

    auth_token = (token_override or token or "").strip()
    if not auth_token:
        raise ValueError("An OpenSky auth token is required to call the REST endpoint.")
    headers = {"Authorization": f"Bearer {auth_token}"}
    params = {
        "airport": airport.upper(),
        "begin": begin_ts,
        "end": end_ts,
    }

    request_session = session or requests.Session()
    try:
        response = request_session.get(
            f"{OPEN_SKY_BASE_URL}/flights/departure",
            params=params,
            headers=headers,
            timeout=timeout,
        )
        response.raise_for_status()
        payload = response.json()
    finally:
        if session is None:
            request_session.close()

    if not isinstance(payload, list):
        raise ValueError("Unexpected response from OpenSky departures endpoint.")

    return payload


In [48]:
def datetime_to_unix(String: datetime) -> int:
    """Convert a datetime or ISO formatted string into a Unix timestamp."""
    if isinstance(String, datetime):
        dt = String
    elif isinstance(String, str):
        value = String.strip()
        if not value:
            raise ValueError("datetime string is empty")
        normalized = value[:-1] + "+00:00" if value.endswith("Z") else value
        try:
            dt = datetime.fromisoformat(normalized)
        except ValueError as exc:
            try:
                dt = datetime.strptime(value, "%Y-%m-%d %H:%M:%S")
            except ValueError as fallback_exc:
                raise ValueError(f"Unsupported datetime string: {String}") from fallback_exc
    else:
        raise TypeError("datetime_to_unix expects a datetime or str instance")
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    else:
        dt = dt.astimezone(timezone.utc)
    return int(dt.timestamp())


In [ ]:
start = datetime_to_unix("2025-08-17 13:00:00")
end = datetime_to_unix("2025-08-17 15:00:00")
# data = api.get_flights_from_interval(1517227200, 1517230800)
s = api.get_states()
print(s)  # Print the retrieved states for debugging purposes.

None


In [60]:
start = datetime_to_unix("2025-08-17 13:00:00")
end = datetime_to_unix("2025-08-17 15:00:00")
get_departures("EDDF", 1517227200, 1517230800)

[{'icao24': '3c4ad0',
  'firstSeen': 1517230790,
  'estDepartureAirport': 'EDDF',
  'lastSeen': 1517238306,
  'estArrivalAirport': None,
  'callsign': 'DLH630  ',
  'estDepartureAirportHorizDistance': 4319,
  'estDepartureAirportVertDistance': 65,
  'estArrivalAirportHorizDistance': None,
  'estArrivalAirportVertDistance': None,
  'departureAirportCandidatesCount': 1,
  'arrivalAirportCandidatesCount': 0},
 {'icao24': '3c5469',
  'firstSeen': 1517230716,
  'estDepartureAirport': 'EDDF',
  'lastSeen': 1517233481,
  'estArrivalAirport': 'LSGG',
  'callsign': 'DLH5LA  ',
  'estDepartureAirportHorizDistance': 4362,
  'estDepartureAirportVertDistance': 103,
  'estArrivalAirportHorizDistance': 1803,
  'estArrivalAirportVertDistance': 163,
  'departureAirportCandidatesCount': 1,
  'arrivalAirportCandidatesCount': 2},
 {'icao24': '4baa03',
  'firstSeen': 1517230625,
  'estDepartureAirport': 'EDDF',
  'lastSeen': 1517238702,
  'estArrivalAirport': None,
  'callsign': 'THY9AD  ',
  'estDeparture